# Прием и обработка твитов микробатчем

## Инициализация

In [ ]:
import org.apache.spark.sql.types.{StructType, StringType, IntegerType, TimestampType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Row
import org.apache.toree.kernel.api
import java.util.Calendar

In [ ]:
println(s"Current spark version is ${spark.version}")

## Чтение модели

In [ ]:
val modelPath = "/home/jovyan/models/spark-ml-model"
val model = PipelineModel.load(modelPath)

## Определяем схему и инициируем потоковый датафрейм

In [ ]:
val inputStreamPath = "/home/jovyan/work/events-stream"
//val outputStreamPath = "/home/jovyan/work/events-stream-out"

// Определяем udf для получения probability по 0 и 1
val getProbability =
    udf(
        (prediction: org.apache.spark.ml.linalg.Vector, pos: Integer) =>
        {
            prediction(pos)
        }
    )

val dataSchema = new StructType()
    .add("tweet", StringType)
    .add("hiddentargetclue", IntegerType)
    .add("timestamp", TimestampType)

In [ ]:
/*
                    .select(
                        $"arrived_key",
                        $"tweet",
                        // $"hiddentargetclue",
                        (getProbability($"probability",lit(0))).alias("Negative Probability")
                    )
*/

/*
                model.transform(batchDF)
                    .select(
                        $"timestamp",
                        $"tweets"
                    )
*/

val inputDF = spark
    .readStream
    .schema(dataSchema)
//    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)
    .withWatermark("timestamp", "15 seconds")
    .groupBy(
        window($"timestamp", "10 seconds", "5 seconds").alias("timestamp")
    )
    .count()
    //.agg(count("*")).alias("tweets")

//val wStream = inputDF.writeStream
//    .option("checkpointLocation", outputStreamPath)
//.start(outputStreamPath)

## Микробатч приема твитов

In [ ]:
var fRuns = 0
var fTweets:Long = 0

// Микробатч для вывода результата предсказания
// Выводится вероятность негативного твита
// В задании написано, что это последняя колонка, но она здесь вроде первая (в позиции 0)
val stream = inputDF.writeStream.foreachBatch {
    (batchDF: DataFrame, batchId: Long) => {
        try {
            fRuns += 1
            //batchDF.foreach {
            //    row:Row => {
            //        fTweets += 1
            //        //row.getAs("tweets")
            //    }
            //}
            batchDF.show()
            //print(s"${Calendar.getInstance().toInstant} - loaded $fTweets rows from the events stream $fRuns times"+13.toChar)
            // Применяем модель и получаем соотв. датасет с предсказаниями
//            globDF = 
//                model.transform(batchDF)
//                    .select(
//                        $"timestamp",
//                        $"tweets"
//                    )
            
        } catch {
            case e:Throwable => {
                println(e.getMessage)
            }
        }
    }
}.start()

## Останов чтения потока

In [ ]:
stream.stop()